# 15-01 跨平台开发

Electron 和 React Native 让 Web 开发者能构建桌面和移动应用。

## 1. Electron 桌面应用

In [ ]:
// main.js - Electron 主进程
const { app, BrowserWindow } = require('electron');
const path = require('path');

function createWindow() {
  const win = new BrowserWindow({
    width: 1200,
    height: 800,
    webPreferences: {
      nodeIntegration: true,
      contextIsolation: false
    }
  });
  
  // 加载本地文件或远程 URL
  win.loadFile('index.html');
  // win.loadURL('https://myapp.com');
}

app.whenReady().then(createWindow);

app.on('window-all-closed', () => {
  if (process.platform !== 'darwin') app.quit();
});

## 2. 主进程与渲染进程通信

In [ ]:
// preload.js
const { contextBridge, ipcRenderer } = require('electron');

// 安全地暴露 API 给渲染进程
contextBridge.exposeInMainWorld('electronAPI', {
  openFile: () => ipcRenderer.invoke('dialog:openFile'),
  onUpdateMessage: (callback) => ipcRenderer.on('update-message', callback)
});

// main.js
const { ipcMain, dialog } = require('electron');

ipcMain.handle('dialog:openFile', async () => {
  const result = await dialog.showOpenDialog({
    properties: ['openFile']
  });
  return result.filePaths;
});

// renderer.js (前端)
const btn = document.getElementById('btn');
btn.addEventListener('click', async () => {
  const filePath = await window.electronAPI.openFile();
  console.log('Selected:', filePath);
});

## 3. React Native 移动应用

In [ ]:
// App.js
import React, { useState } from 'react';
import { View, Text, TextInput, Button, StyleSheet } from 'react-native';

export default function App() {
  const [message, setMessage] = useState('');
  const [response, setResponse] = useState('');
  
  const sendMessage = async () => {
    const res = await fetch('https://api.example.com/chat', {
      method: 'POST',
      body: JSON.stringify({ message })
    });
    const data = await res.json();
    setResponse(data.reply);
  };
  
  return (
    <View style={styles.container}>
      <Text style={styles.title}>AI Chat</Text>
      <TextInput
        style={styles.input}
        placeholder="输入消息..."
        value={message}
        onChangeText={setMessage}
      />
      <Button title="发送" onPress={sendMessage} />
      {response && <Text style={styles.response}>{response}</Text>}
    </View>
  );
}

const styles = StyleSheet.create({
  container: { flex: 1, padding: 20, paddingTop: 50 },
  title: { fontSize: 24, fontWeight: 'bold', marginBottom: 20 },
  input: { borderWidth: 1, padding: 10, marginBottom: 10, borderRadius: 5 },
  response: { marginTop: 20, padding: 10, backgroundColor: '#f0f0f0' }
});

## OpenClaw 的跨平台架构

```
OpenClaw 项目结构：
├── src/           # 核心（TypeScript/Node.js）
├── apps/
│   ├── ios/       # Swift/iOS
│   ├── android/   # Kotlin/Android
│   └── macos/     # Swift/macOS
└── ui/            # 共享 UI 组件
```

## 练习

1. 创建一个简单的 Electron 应用
2. 搭建 React Native 开发环境
3. 查看 OpenClaw 的移动端实现